## Context

This notebook is an end-to-end example that you can follow to create a project, upload a first plugin and activate it on this project, and later see the plugin in action and monitor it.

**NB: The plugin capabilities of Kili are under active development, and compatible with version 2.125.2 & superior of kili. don't hesitate to reach out via github or the support to provide feedbacks.**


## Step 1 : Instantiate Kili with your API_KEY

In [ ]:
!pip install --upgrade kili

In [ ]:
from kili.client import Kili
import os
import time
import json
import glob

kili = Kili()

## Step 2 : Create the project

First, we need to create a new project. For our example, it will be a project of type `IMAGE` and it will have the following `jsonInterace` :

In [ ]:
# Create the project

title = "Programmatic QA - Invoices processing"
description = "Programmatic QA - Invoices processing"
input_type = "IMAGE"


filename = "./datasets/document_plugin_data/jsoninterface.json"

with open(filename, "r") as f:
    json_interface = json.load(f)

project = kili.create_project(
    title=title, description=description, input_type=input_type, json_interface=json_interface
)

project_id = project["id"]

In [ ]:
# Add assets

content_array = [
    "https://storage.googleapis.com/label-public-staging/doc-plugins/1502519745_VFT024147517-1.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/734-16a4595dbfa7f0126f0ddcaa54c6531ae554356d.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/document_data.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/facture%20.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/fcl091709145134-1.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/images.png",
    "https://storage.googleapis.com/label-public-staging/doc-plugins/modele_de_facture-1.png",
]
names_array = [
    "document1",
    "document2",
    "document3",
    "document4",
    "document5",
    "document6",
    "document7",
]


project = kili.append_many_to_dataset(
    project_id=project_id, content_array=content_array, external_id_array=names_array
)


kili.update_properties_in_project(
    project_id=project["id"],
    metadata_types={
        "accuracy": "number",
    },
)

This project has two jobs with several tasks of transcription associated. We will be interested in the *Payment information* job, and more precisely the *IBAN* and the *Currency* sub-jobs. With our plugin, we want to be sure that the labelers write the correct values for the two transcriptions (so we are interested only in the `on_submit` handler of the plugin), since we know for sure that the IBAN should start with the 2 letters *FR*, and that the currency should be one of : *EURO*, *DOLLAR*. At then end, we also calculate an accuracy and we insert it in the `json_metadata` of the asset.

To iterate on the plugin code, you can refer to *plugins_development.ipynb* notebook


```python
from kili.plugins import PluginCore
from typing import Dict


class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def check_rules_on_label(self, label: Dict):

        issues_array = []
        mid_issues_array = []

        for bbox in label["jsonResponse"]["JOB_0"]["annotations"]:

            # Rule 1 - Check IBAN starts by FR
            if bbox["categories"][0]["name"] == "IBAN":
                iban = bbox["children"]["TRANSCRIPTION_JOB"]["text"]

                if iban[0:2] != "FR":
                    issues_array.append("IBAN number should start by FR")
                    mid_issues_array.append(bbox["mid"])

            # Rule 2 - Check if Currency is in list of fields
            if bbox["categories"][0]["name"] == "CURRENCY":
                currency = bbox["children"]["TRANSCRIPTION_JOB_2"]["text"]

                if currency not in ["DOLLAR", "EURO"]:
                    issues_array.append("Authorized currency are only Euro and Dollar")
                    mid_issues_array.append(bbox["mid"])

        return issues_array, mid_issues_array

    def on_submit(self, label: Dict, asset_id: str) -> None:
        """
        Dedicated handler for Submit action
        """
        self.logger.info("On submit called")

        issues_array, mid_issues_array = self.check_rules_on_label(label)

        project_id = self.project_id

        if len(issues_array) > 0:
            print("Creating an issue...")

            self.kili.create_issues(
                project_id=project_id,
                label_id_array=[label['id']] * len(issues_array),
                object_mid_array=mid_issues_array,
                text_array=issues_array,
            )

            print("Issue created!")

            self.kili.add_to_review(asset_ids=[asset_id])

            print("Asset added to review")

        accuracy = (
            100 - len(issues_array) / len(label["jsonResponse"]["JOB_0"]["annotations"]) * 100
        )

        print(accuracy)
        self.kili.update_properties_in_assets(
            asset_ids=[asset_id], json_metadatas=["{'accuracy': accuracy}"]
        )

        print("Accuracy score computed")


## Step 3 : Uploading the plugin

With the plugin defined in a separate `python` file, we can now upload it and activate it on our project.

- the upload will create the necessary builds to execute the plugin, it will take a few minutes.
- After the activation, you can start using it right away.

In [ ]:
path_to_plugin = "./plugins_library/plugin_document.py"
plugin_name = "Plugin - Programmatic QA invoices"

kili.upload_plugin(path_to_plugin, plugin_name)

kili.activate_plugin_on_project(plugin_name, project_id=project_id)